in this notebook i'm trying to get renames for all (active and inactive) tickers.

what i've found so far.

1) there are a lot of cases when polygon doesn't provide figi for a ticker. and this happens not only for tickers that 
    were desilted before a figi was assigned. it happens even for active companies where tradingview shows their figi.
    for example, XP BBG00QVJYGM9 is trading already 5 years starting from 2020. the ticker XP is present but no figi.

2) for 1 figi sometimes there are more than one ticker at the same time. some examples:

    2025-05-30,VLGE.A,BBG000BWGK40,True,          ,usd,,2016-05-18T00:00:00Z,us,stocks,VILLAGE SUPER MARKET CL-A NEW,XNAS,
    2025-05-30,VLGEA ,BBG000BWGK40,True,0000103595,usd,,2025-05-30T00:00:00Z,us,stocks,Village Super Market         ,XNAS,CS

    here tickers will be the same if '.' is removed but look at the next case:

    2012-04-24,ABD  ,BBG000J06K07,True,0000712034,usd,,2012-04-26T00:00:00Z,us,stocks,ACCO BRANDS CORPORATION,XNYS,CS
    2012-04-24,ACCOw,BBG000J06K07,True,0000712034,usd,,2012-04-30T00:00:00Z,us,stocks,ACCO BRANDS CORPORATION W.I.,XNYS,

    usually warrants are with 'w' suffix but here the whole ticker is different.

3) sometimes there are complete duplicates. for example, https://api.polygon.io/v3/reference/tickers?ticker=UST&market=stocks&date=2025-06-01&active=true&order=asc&limit=100&sort=ticker&apiKey= returns:
    {
    "results": [
        {
        "ticker": "UST",
        "name": "ProShares Ultra 7-10 Year Treasury",
        "market": "stocks",
        "locale": "us",
        "primary_exchange": "ARCX",
        "type": "ETF",
        "active": true,
        "currency_name": "usd",
        "composite_figi": "BBG000BH4371",
        "share_class_figi": "BBG001SK30C5",
        "last_updated_utc": "2024-09-23T00:00:00Z"
        },
        {
        "ticker": "UST",
        "name": "ProShares Ultra 7-10 Year Treasury",
        "market": "stocks",
        "locale": "us",
        "primary_exchange": "ARCX",
        "type": "ETF",
        "active": true,
        "currency_name": "usd",
        "cik": "0001373525",
        "composite_figi": "BBG000BH4371",
        "share_class_figi": "BBG001SK30C5",
        "last_updated_utc": "2025-06-02T00:00:00Z"
        }
    ],
    "status": "OK",
    "request_id": "54d441e0bb407fc21739f13b8a5af347",
    "count": 2
    }

    the only difference is cik.



In [1]:
import pandas as pd
import sys
import os
# Add the parent directory to Python path to import api_key module
sys.path.append(os.path.dirname(os.path.abspath('')))
import settings

In [2]:
tickers_dir = os.path.join(settings.ABSOLUTE_DATA_DIR, 'tickers')
active_tickers_file = os.path.join(tickers_dir, 'tickers_history_active.csv')
inactive_tickers_file = os.path.join(tickers_dir, 'tickers_history_inactive.csv')

In [ ]:
import csv
from datetime import datetime, timedelta

state = {}
history = {}
counter = 0

with open(active_tickers_file, newline='', encoding='utf-8') as fin:
    reader = csv.DictReader(fin)
    for row in reader:
        counter += 1
        if counter % (1000 * 1000) == 0:
            print(f"Processed {counter} rows")
        figi  = row["composite_figi"]
        if not figi:
            # Skip rows without FIGI
            continue
        ticker= row["ticker"]
        date  = datetime.fromisoformat(row["date"]).date()
        if figi not in state:
            # first time encountering this FIGI
            state[figi] = { 'ticker': ticker, "start": date }
            history[figi] = { ticker: { 'start' : date, 'end': None } }
        else:
            # known FIGI, check if ticker changed
            prev_ticker = state[figi]['ticker']
            # if ticker changed - update state and history
            if ticker != prev_ticker:
                fixed_ticker = ticker.replace('.', '').replace('/', '')
                fixed_prev_ticker = prev_ticker.replace('.', '').replace('/', '')
                if fixed_ticker != fixed_prev_ticker\
                    and not fixed_ticker.startswith(fixed_prev_ticker)\
                    and not fixed_prev_ticker.startswith(fixed_ticker):
                    if ticker in history[figi]:
                        print(f'error: ticker already exists in history for FIGI: {figi}, ticker: {ticker}, prev_ticker: {prev_ticker}, date: {date}')
                    # save the end date for the previous ticker
                    end_date = date - timedelta(days=1)
                    history[figi][prev_ticker]['end'] = end_date
                    # update state and history with the new ticker
                    state[figi] = {"ticker": ticker, "start": date}
                    history[figi][ticker] = { 'start': date, 'end': None }

history

In [ ]:
history['BBG000MM2P62']

{'FB': {'start': datetime.date(2012, 5, 18), 'end': datetime.date(2022, 6, 8)},
 'META': {'start': datetime.date(2022, 6, 9), 'end': None}}

In [ ]:
surprise = {}

with open(inactive_tickers_file, newline='', encoding='utf-8') as fin:
    reader = csv.DictReader(fin)
    for row in reader:
        counter += 1
        if counter % (1000 * 1000) == 0:
            print(f"Processed {counter} rows")
        figi  = row["composite_figi"]
        if not figi:
            # Skip rows without FIGI
            continue
        ticker= row["ticker"]
        date  = datetime.fromisoformat(row["date"]).date()
        if figi not in history:
            if figi not in surprise:
                surprise[figi] = { ticker : { 'first_time': date, 'unknown_figi': True } }
            else:
                surprise[figi][ticker]['last_time'] = date
            continue
        else:
            history_figi = history[figi]
            if ticker not in history_figi:
                # known FIGI, but new ticker
                if figi not in surprise:
                    surprise[figi] = { ticker: { 'first_time': date, 'unknown_ticker': True } }
                elif ticker not in surprise[figi]:
                    surprise[figi][ticker] = { 'first_time': date, 'unknown_ticker': True }
                else:
                    surprise[figi][ticker]['last_time'] = date
                continue
            else:
                history_figi_ticker = history_figi[ticker]
                if history_figi_ticker['end'] is None:
                    history_figi_ticker['end'] = date
                    history_figi_ticker['delisted_utc'] = row['delisted_utc']
                else:
                    history_figi_ticker['last_inactive'] = date
surprise

ok. so there are several observations here:

1) it happens that there appears a record about inactive ticker with figi while the there were no such figi among active tickers.

i did search manually for BBG00FZLQV86, BBG000F4MYC2, BBG000GQR282 in the active tickers csv and haven't found anything.
while they can be found in the inactive tickers csv.

also this can be proven via https://polygon.io/docs/rest/stocks/tickers/all-tickers
for example, for CNET on 2020-10-13 there is only one active ticker with "last_updated_utc": "2016-05-18T00:00:00Z".
on 2020-10-14 there appears BBG00DP4PXQ7 with "delisted_utc": "2020-10-14T00:00:00Z". looks like there were 4 years without deals for this ticker and figi was not updated until delisting.

2) if the figi is known for a ticker then deactivation record appears always after active records. means to crap here.

3) it happens that for a given figi deactivation records stop to appear. a good example is BBG00GVX5D49 that appeared only once on 
2021-08-02.



In [ ]:
tickers_renames = []
# for each key in history
for figi, tickers in history.items():
    for ticker, dates in tickers.items():
        tickers_renames.append([
            figi,
            ticker,
            dates['start'],
            dates.get('end', None),
            dates.get('last_inactive', None),
            dates.get('delisted_utc', None)
        ])

df = pd.DataFrame(
    tickers_renames,
    columns=[
        'figi',
        'ticker',
        'start_date',
        'end_date',
        'last_inactive',
        'delisted_utc'
    ]
)

# sort by figi and start_date
df.sort_values(by=['figi', 'start_date'], inplace=True)

df.to_csv(
    os.path.join(tickers_dir, 'tickers_renames.csv'),
    index=False
)
